Henning Gu - Matrikelnummer: 81187 - Projekt für Data Analysis und Digital Reporting mit Python

Alle Unternehmen die zum Zeitpunkt des Projektes im Dow Jones sind und setz diese als res.

In [45]:
fmp_url = 'https://financialmodelingprep.com/api/v3'
dj = 'dowjones_constituent'
api_key = '9e7dcdbfc6b35f8a73a1d9e872d2b519'

import requests as req
params = {'apikey': api_key}
res = req.get(fmp_url + '/' + dj, params=params)

 Diverse Imports

In [46]:
from jupyter_dash import JupyterDash
from dash import html, dcc
from dash.dependencies import Input, Output

import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots

import pandas as pd
import pandas_ta as ta

from sklearn.metrics import mean_squared_error
import locale

Setze res als core_data

In [47]:
core_data = pd.DataFrame.from_dict(res.json())

Nimmt das Symbol für alle Unternehmen in den Kerndaten, fügt das Profil des jeweiligen Unternehmens an und erstellt ein neues DataFrame (DF) namens Profil.

In [48]:
profiles = pd.DataFrame()

for sym in core_data.symbol:
    tmp_url = fmp_url + '/profile/' + sym
    data = req.get(tmp_url, params=params).json()
    tmp_df = pd.DataFrame(data=data, index=[0])
    profiles = pd.concat([profiles, tmp_df], ignore_index=True)


Nimmt das Symbol für alle Unternehmen in den Kerndaten, fügt die Umsätze der letzten 4 Quartale des jeweiligen Unternehmens an und erstellt ein neues DataFrame (DF) namens earningsOfTheLastFourQuarters.

In [49]:
earningsOfTheLastFourQuarters = pd.DataFrame()

for sym in core_data.symbol:
    tmp_url = fmp_url + '/income-statement/' + sym + '?period=quarter&limit=4'
    data = req.get(tmp_url, params=params).json()
    tmp_df = pd.DataFrame(data=data)
    earningsOfTheLastFourQuarters = pd.concat([earningsOfTheLastFourQuarters, tmp_df], ignore_index=True)


Nimmt das Symbol für alle Unternehmen in den Kerndaten, fügt die Vorhersagen für des jeweiligen Unternehmens an und erstellt ein neues DataFrame (DF) namens estimates.

In [50]:
estimates = pd.DataFrame()

for sym in core_data.symbol:
    tmp_url = fmp_url + '/analyst-estimates/' + sym + '?period=quarter&limit=16'
    data = req.get(tmp_url, params=params).json()
    tmp_df = pd.DataFrame(data=data)
    estimates = pd.concat([estimates, tmp_df], ignore_index=True)


Nimmt das Symbol für alle Unternehmen in den Kerndaten, fügt die Daten der Aktie seit Jahresbeginn 2021 an und erstellt ein neues DataFrame (DF) namens dailyStock.

In [51]:
dailyStock = pd.DataFrame()

for sym in core_data.symbol:
    tmp_url = fmp_url + '/historical-price-full/' + sym + '?from=2021-01-01'
    data = req.get(tmp_url, params=params).json()
    tmp_df = pd.DataFrame(data=data)
    dailyStock = pd.concat([dailyStock, tmp_df], ignore_index=True)

Nimmt die historischen für den Dow Jones und erstellt ein DateFrame daraus. 

In [52]:
dowJones = pd.DataFrame()

tmp_url = fmp_url + '/historical-price-full/%5EDJI?from=2021-01-01'
data = req.get(tmp_url, params=params).json()
tmp_df = pd.DataFrame(data=data)
dowJones = pd.concat([dowJones, tmp_df], ignore_index=True)

Verbindet die Tabellen Kerndaten und profiles anhand des Symbols. Anschließend wird sie nach dem Symbol sortiert und in eine neue Tabelle übertragen.

In [53]:
tempdf_profiles = core_data.merge(profiles, how="inner", on="symbol")
df_profiles = tempdf_profiles.sort_values(by='symbol')

Setzt den 31.12.2021 in UNIX Zeit als this_quarter, wird für die Erwartungen benötigt.

In [54]:
from datetime import datetime
timestamp = 1640991000 # Entspricht 23:50 am 31.12.2021
date_time = datetime.fromtimestamp(timestamp)
this_quarter = date_time.strftime("%Y-%m-%d") # 2021-12-31
print(this_quarter)

2021-12-31


Verbindet die Tabellen Kerndaten und Estimates anhand des Symbols.
Anschließend wird die entstandene Tabelle nach dem Jahr gefiltert. Einmal für 2021 & 2022 um historische Vorhersagen zu bekommen, einmal für 2022 & 2023 um Vorhersagen für dieses und das nächste Jahr zu bekommen.
Aus diesen neuen Tabellen werden die Name der Unternehmen, das Datum, der niedrigste, mittlere und der höchste EBITDA sowie der Anzahlen der Vorhersagen gefiltert, welche dann nach dem Namen und dem Datum sortiert.

In [55]:
tempdf_estimates = core_data.merge(estimates, how="inner", on="symbol")

df_estimatesWithHistory = tempdf_estimates.loc[
    tempdf_estimates.loc[:,"date"].str.contains(r'(?:2021|2022)'), 
    ['name', 'date', 'estimatedEbitdaLow', 'estimatedEbitdaAvg', 'estimatedEbitdaHigh', 'numberAnalystEstimatedRevenue']
    ].sort_values(['name', 'date'])

df_estimatesWithFuture = tempdf_estimates.loc[
    tempdf_estimates.loc[:,"date"].str.contains(rf'(?:2022|2023)|{this_quarter}'), 
    ['name', 'date', 'estimatedEbitdaLow', 'estimatedEbitdaAvg', 'estimatedEbitdaHigh', 'numberAnalystEstimatedRevenue']
    ].sort_values(['name', 'date'])

Verbindet die Tabellen Kerndaten und Earnings anhand des Symbols.
Aus der resultierenden Tabelle werden die Name der Unternehmen, das Datum des Eintrags, das betroffene Jahr, das jeweilige Quartal und der Ebitda genommen, welche dann nach dem Namen, dem Jahr und anschließend der Periode sortiert.

In [56]:
tempdf_earnings = core_data.merge(earningsOfTheLastFourQuarters, how="inner", on="symbol")

df_earnings_history = tempdf_earnings.loc[:, 
    ['name', 'date', 'calendarYear', 'period', 'ebitda']
].sort_values(['name', 'calendarYear', 'period'])

Verbindet die Tabellen Kerndaten und dailyStock anhand des Symbols. 

In [57]:
dailyStock = core_data.merge(dailyStock, how="inner", on="symbol")

 Layout der App

In [487]:
app = JupyterDash()

app.layout = html.Div([
    html.H1(children='Abschlussprojekt', style={'textAlign': 'center', 'fontFamily':'Monospace'}),
    html.P(
            [
                dcc.Dropdown(id='company_dd',
                             options=[{'label': i, 'value': i}
                                      for i in df_profiles.name.unique()],
                             value=df_profiles.name.unique()[0],
                             style=dict(width='300px', marginTop='10px', marginBottom='10px', marginLeft='auto', marginRight='auto', fontFamily='Monospace')),

                html.H2(id='company_info', style={'textAlign': 'center', 'fontFamily':'Monospace'}),

                dcc.Graph(id='lineplot_earnings'),

                html.P(id='earnings_prediction_mse', style={'textAlign': 'end', 'fontSize': '24px', 'fontFamily':'Monospace'}),

                dcc.Graph(id='lineplot_estimates'),

                dcc.Graph(id='lineplot_stockPrice'),

                dcc.Checklist(
                    id='add_dj',
                    options=[
                        {'label': 'Den Dow Jones Index hinzufügen', 'value': 'dj'},
                    ],
                    style={'textAlign': 'center', 'fontFamily':'Monospace', 'fontSize': '20px', 'marginTop': '25px', 'marginBottom': '25px'}
                ),

                dcc.Checklist(
                    id='add_ols',
                    
                    options=[
                        {'label': 'Den OLS hinzufügen', 'value': 'ols'},
                    ],
                    style={'textAlign': 'center', 'fontFamily':'Monospace', 'fontSize': '20px', 'marginTop': '25px', 'marginBottom': '25px'}
                ),
                
                dcc.Graph(id='lineplot_RSI'),
            ]
    )
])


Funktion für den Graph des Umsatzes.
Filtert die Tabellen für den Umsatz und der Vorhersagen bis zum aktuellen Tag nach dem Unternehmen, der im Dropdown ausgewählt wird. Die mittlere quadratische Abweichung wird anhand der Ebitda und der durchschnittlich vorhergesagten Ebitda (die für das aktuelle und nächste Jahr) für die jeweiligen Unternehmen berechnet.

Dann werden die Graphen gesetzt.

Die mittlere quadratische Abweichung wird in wissenschaftlicher Notation gerundet auf vier Nachkommastellen wiedergegeben.

In [488]:
@app.callback(
    Output(component_id='lineplot_earnings', component_property='figure'),
    Output('earnings_prediction_mse', 'children'),
    Input('company_dd', 'value'),
)

def outfigure(company):
    locale.setlocale(locale.LC_ALL, 'de_DE')
    earningsForPlot = df_earnings_history[df_earnings_history.name == company]
    estimatesForPlot = df_estimatesWithHistory[df_estimatesWithHistory.name == company]
    mseForPlot = mean_squared_error(earningsForPlot.ebitda, estimatesForPlot[:4].estimatedEbitdaAvg)
    fig = px.line(
        earningsForPlot, 
        x='date', 
        y='ebitda', 
        title="EBITDA (Gewinn nach Steuern, Zinsen und Abschreibungen)", 
        markers=True,
        )
        
    fig['data'][0]['showlegend'] = True
    fig['data'][0]['name'] = "EBITDA"

    fig.add_scatter(x=estimatesForPlot.date, y=estimatesForPlot.estimatedEbitdaAvg, name="Vorhergesagte EBITDA")
    
    fig.update_layout(
        plot_bgcolor='#efefef',
        font_family='Monospace',
        font_color='#000000',
        font_size=20,
        yaxis_title='EBITDA', 
        xaxis_title='Datum', 
        hovermode="x unified",
    )

    fig.update_xaxes(
        ticktext=["Q2 2021", "Q3 2021", "Q4 2021", "Q1 2022", "Q2 2022", "Q3 2022", "Q4 2022", "Q1 2023"],
        tickvals=["2021-04-01", "2021-07-01", "2021-10-01", "2022-01-01", "2022-04-01", "2022-07-01", "2022-10-01", "2023-01-01"],
    )
    fig.update_yaxes(
        tickformat="$.4s"
    )

    fig.update_traces(line=dict(width=4), marker={'size': 10})
    
    return (fig, html.P(f"MQA: {mseForPlot:.2E}".replace('.',',')))

Funktion für den Graph der Vorhersagen.
Filtert die Tabelle nach den Vorhersagen für das Unternehmen, welcher im Dropdown ausgewählt wird. 
Es werden zwei Graphen angezeigt, einmal für die niedrige und einmal für die höhere Vorhersage.

In [489]:
@app.callback(
    Output(component_id="lineplot_estimates", component_property="figure"),
    Input("company_dd", "value"),
)

def outfigure(company):
    estimatesForPlot = df_estimatesWithFuture[df_estimatesWithFuture.name == company]
    fig = px.line(
        estimatesForPlot, 
        x="date", 
        y=["estimatedEbitdaLow", "estimatedEbitdaHigh"], 
        title="Erwartete EBITDA",
        hover_data=["numberAnalystEstimatedRevenue"],
        markers=True,
        )

    fig.update_layout(
        plot_bgcolor="#efefef",
        font_family="Monospace",
        font_color="#000000",
        font_size=20,
        showlegend=True, 
        legend_title_text=f"{company}", 
        yaxis_title="Erwartete EBITDA", 
        hovermode="x unified"
        )
    
    fig.update_xaxes(title_text="Datum")
    
    fig.update_traces(line=dict(width=4), marker={'size': 10})

    return fig

Funktion für den Graph der Tagesschlusskurse.
Diesen kann der Dow Jones hinzugefügt werden, der dann eine seperate Y-Achse benutzt.

In [490]:
@app.callback(
    Output(component_id='lineplot_stockPrice', component_property='figure'),
    Input('company_dd', 'value'),
    Input('add_dj', 'value'),
    Input('add_ols', 'value'),
)

def outfigure(company, dj, ols):
    LaLaLa = dailyStock.loc[:,["name", "historical"]]
    Bla = LaLaLa[LaLaLa.name==company].historical
    stockPriceForPlot = pd.DataFrame.from_records(Bla.reset_index(drop=True))
    
    mittelwert = sum(stockPriceForPlot['close'])/len(stockPriceForPlot['close'])
    
    dowJonesGraph = pd.DataFrame.from_records(dowJones.historical)
    
    # Schlusskurs der jeweiligen Aktie mit dem Schlusskurs des Dow Jones
    if dj:
        fig = make_subplots(specs=[[{"secondary_y": True}]])

        fig.add_trace(
            go.Scatter(x=stockPriceForPlot['date'], y=stockPriceForPlot['close'], name=f'{company}', line=dict(width=4)),
            secondary_y=False)

        fig.add_trace(
            go.Scatter(x=dowJonesGraph['date'], y=dowJonesGraph['close'], name="Dow Jones", line=dict(width=4)),
            secondary_y=True)

        fig.update_layout(
            title_text="Der tägliche Schlusskurs und der DJI"
        )

        fig.update_xaxes(title_text="Datum")

        fig.update_yaxes(title_text="Schlusskurs (in $)", tickformat="$", secondary_y=False)
        # fig.update_yaxes(title_text="Dow Jones", secondary_y=True)
        fig.update_yaxes(title_text="Dow Jones", tickformat="$.4s", secondary_y=True)

        fig.update_layout(
            hovermode="x unified", 
            plot_bgcolor='#efefef',
            font_family='Monospace',
            font_color='#000000',
            font_size=20
            )

        return fig

    #####################################################################################################################################################
    # Hier wusste ich nicht, anhand welcher Daten man die Regression berechnen sollte. 
    # Wegen der Überschneidung des Projekts mit einer weiteren Klausur habe ich dies nicht fertigstellen können
    
    elif ols:
        fig = make_subplots(specs=[[{"secondary_y": True}]])

        fig.update_layout(xaxis2= {'anchor': 'y', 'overlaying': 'x', 'side': 'top'},
                  yaxis_domain=[0, 0.94])

        fig.add_trace(
            go.Scatter(x=stockPriceForPlot['date'], y=stockPriceForPlot['close'], name=f'{company}', line=dict(width=4)),
            secondary_y=False)

        fig.add_trace(
            go.Scatter(x=dowJonesGraph['date'], y=dowJonesGraph['close'], name="Dow Jones", line=dict(width=4)),  
            secondary_y=True)


        fig.add_trace(
            go.Scatter(x=dowJonesGraph['close'], y=stockPriceForPlot['close'], name="OLS", line=dict(width=4)),
            secondary_y=False)

        
        fig.update_layout(
            title_text="Der tägliche Schlusskurs und der DJI"
        )

        fig.update_xaxes(title_text="Datum")

        fig.update_yaxes(title_text="Schlusskurs (in $)", secondary_y=False)
        fig.update_yaxes(title_text="Dow Jones", secondary_y=True)

        fig.update_layout(
            hovermode="x unified", 
            plot_bgcolor='#efefef',
            font_family='Monospace',
            font_color='#000000',
            font_size=20
            )
        
        fig.data[2].update(xaxis='x2')
        return fig

    #####################################################################################################################################################

    # Schlusskurs der jeweiligen Aktie
    else:
        fig = px.line(
            stockPriceForPlot, 
            x='date', 
            y='close',
            title="Täglicher Schlusskurs",
            )

        fig.update_layout(
            plot_bgcolor='#efefef',
            font_family='Monospace',
            font_color='#000000',
            font_size=20,
            legend_title_text=f'{company}', 
            yaxis_title='Schlusskurs', 
            hovermode="x unified"
            )

        fig.update_traces(line=dict(width=4))

        fig.update_xaxes(title_text="Datum")

        fig.update_yaxes(tickformat="$")
        
        return fig

Funktion für den Graph des RSI. 
Hier wurde Pandas Ta benutzt, um den RSI zu berechnen.

In [491]:
@app.callback(
    Output(component_id='lineplot_RSI', component_property='figure'),
    Input('company_dd', 'value'),
)

def outfigure(company):
    temp = dailyStock.loc[:,["name", "historical"]]
    temp2 = temp[temp.name==company].historical
    
    stockPriceForRSI = pd.DataFrame.from_records(temp2.reset_index(drop=True))

    # Erstellt eine Spalte names Periode und füllt sie mit 1 bis zur Länge der Tabelle
    stockPriceForRSI['period'] = range(1, len(stockPriceForRSI) + 1)

    rsi = ta.rsi(stockPriceForRSI["close"], append=True)
    rsi = rsi.to_frame()

    # Erstellt eine Spalte names Periode und füllt sie mit 1 bis zur Länge der Tabelle
    rsi['period'] = range(1, len(rsi) + 1)
    
    # Verbindet rsi und stockPriceForRSI anhand der period Spalte miteinander
    rsi = rsi.merge(stockPriceForRSI, how='left', on='period')

    fig = make_subplots(rows=2, cols=1, shared_xaxes=True, row_width=[0.25, 0.75])

    # Graph für den RSI
    fig.add_trace(go.Scatter(
        x=rsi.date,
        y=rsi['RSI_14'],
        line=dict(color='#ff9900', width=4),
        showlegend=False,
        )
    )

    fig.update_yaxes(title_text="RSI", range=[-10, 110])
    fig.update_xaxes(title_text="Datum")

    fig.add_hline(y=0, line_color="#666", line_width=2)
    fig.add_hline(y=100, line_color="#666", line_width=2)

    # Grenzlinien
    fig.add_hline(y=30, line_color='#336699', line_width=2, line_dash='dash')
    fig.add_hline(y=70, line_color='#336699', line_width=2, line_dash='dash')
    
    layout = go.Layout(
        plot_bgcolor='#efefef',
        font_family='Monospace',
        font_color='#000000',
        font_size=20,
        title="Relative Strength Index (RSI)",
    )
    
    fig.update_layout(layout, hovermode="x unified")

    return fig

 Callback und Funktion für die Unternehmensinformationen

In [492]:
@app.callback(
    Output(component_id='company_info', component_property='children'),
    Input('company_dd', 'value'),
)

def company_info(company):
    company_info = df_profiles.loc[df_profiles.name == company]

    return (
        html.P(
            [
                f"{company_info['companyName'].item()}", 

                html.Br(), 

                html.Img(src=company_info['image'].item()),

                html.Br(), 

                f"Mitarbeiter: {company_info['fullTimeEmployees'].item()}",
                
                html.Br(), 

                f"Standort: {company_info['headQuarter'].item()}",

                html.Br(), 

                f"Preis: {company_info['price'].item()}".replace('.',','),
         
                html.Br(), 
                
                f"Marktkapitalisierung: {company_info['mktCap'].item():,}".replace(',','.')
            ]
        )
    )

 Starten der App

In [493]:
# Ohne dies würde die App in Jupyter nicht laufen
del app.config._read_only["requests_pathname_prefix"]

app.run_server()

Dash app running on http://127.0.0.1:8050/


C:\Users\henni\AppData\Local\Programs\Python\Python310\lib\site-packages\jupyter_dash\jupyter_app.py:139: UserWarning:

The 'environ['werkzeug.server.shutdown']' function is deprecated and will be removed in Werkzeug 2.1.

